In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO
from functools import reduce
import operator

## Retrieve Data from Bulbapedia

In [2]:
url = "https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_availability"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
tables = soup.find_all('table', {'class': 'roundy'})

In [4]:
generation_dataframes = {}
for i, table in enumerate(tables):
    table_string = str(table)
    generation_dataframes[i] = pd.read_html(StringIO(table_string))[0]

# This table has a couple extra columns that we don't need
generation_dataframes[7] = generation_dataframes[7].drop(columns=[('Game', 'Unnamed: 12_level_1'),
            ('Game', 'Unnamed: 13_level_1')], axis=1)

In [5]:
# Assuming `generation_dataframes` is your dictionary of dataframes
dfs = list(generation_dataframes.values())
legend = dfs.pop(0)
# Join the dataframes vertically and fill missing columns with NA
merged_df = pd.concat(dfs, axis=0, join='outer', keys=list(generation_dataframes.keys()), sort=False).fillna('—')

/var/folders/67/9rpqzr212xbc567t79k0w7580000gn/T/ipykernel_67115/2615835790.py:5: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  merged_df = pd.concat(dfs, axis=0, join='outer', keys=list(generation_dataframes.keys()), sort=False).fillna('—')


In [6]:
new_column_names = ['Red', 'Green', 'Blue', 'Yellow', 'Gold', 'Silver', 'Crystal', 'Ruby', 'Sapphire', 'FireRed', 'LeafGreen', 'Emerald', 'Colosseum', 'XD', 'Diamond', 'Pearl', 'Platinum', 'HeartGold', 'SoulSilver', 'Black', 'White', 'Black 2', 'White 2', 'X', 'Y', 'Omega Ruby', 'Alpha Sapphire', 'Sun', 'Moon', 'Ultra Sun', 'Ultra Moon', "Let's Go, Pikachu!", "Let's Go, Eevee!", 'Sword', 'Shield', 'Brilliant Diamond', 'Shining Pearl', 'Legends: Arceus', 'Scarlet', 'Violet']


# Assuming `df` is your DataFrame
df = merged_df

# Step 1: Extract current MultiIndex levels and labels
current_levels = df.columns.levels
current_labels = df.columns.get_level_values(0).tolist()


# Step 2 & 3: Create a mapping for generations and their subcategories
generation_mapping = {}
for col in df.columns:
    if col[0] == 'Game':
        main_generation = col[1].split('.')[0]  # Extract main generation
        if main_generation not in generation_mapping:
            generation_mapping[main_generation] = []
        generation_mapping[main_generation].append(col[1])

# Step 4: Create new MultiIndex including the new generation level
new_columns = []
for col in df.columns:
    if col[0] == 'Game':
        main_generation = col[1].split('.')[0]
        new_columns.append(('Game', main_generation, new_column_names.pop(0)))
    else:
        new_columns.append((col[0], '', col[1]))  # Keep other columns unchanged

# Convert list of tuples to MultiIndex
new_multiindex = pd.MultiIndex.from_tuples(new_columns, names=['Category', 'Generation', 'Subcategory'])

# Step 5: Assign the new MultiIndex to the DataFrame
df.columns = new_multiindex
df.drop(columns='Icon', inplace=True)


/var/folders/67/9rpqzr212xbc567t79k0w7580000gn/T/ipykernel_67115/3039264149.py:35: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns='Icon', inplace=True)


In [7]:
legend = legend.droplevel(1, axis=1).drop(index=11)
legend['Availability'] = [1 if i < 11 or i == 18 else 0 for i in range(len(legend))]

In [8]:
def process_symbols(input_string):
    # Define the vectors based on the description
    unique = {"PW", "DR", "DW", "FS", "DA", "EV", "Ev", "CD"}  # Multi-character unique symbols without 'CC'
    both = {"C", "D", "E", "T", "S", "R", "T", "B", "—"}  # Common between certain groups
    suffix_list = ["E", "B", "D", "T"]  # Order matters here
    all_symbols = sorted(unique | both | set(suffix_list) | {"CC"})  # All unique symbols sorted for index retrieval

    # Initialize result list
    results = []
    split = []
    i = 0  # To keep track of the index in the input_string

    # Handle "CC" prefix only at the start of the string
    if input_string.startswith("—"):
        results.append(legend[legend["Symbol"]=="—"].index[0])  # Add index of "CC"
        split.append(["—"])  # Add "-" to the split list
        i += 1  # Skip the "-" at the beginning
    elif input_string.startswith("CC"):
        results.append(legend[legend["Symbol"]=="CC"].index[0])  # Add index of "CC"
        split.append(["CC"])  # Add "CC" to the split list
        i += 2  # Skip the "CC" at the beginning
    # Iterate over each remaining character in the input string
    while i < len(input_string):
        # Check for two-character unique symbols first
        if i < len(input_string) - 1:
            two_char_symbol = input_string[i:i+2]
            if two_char_symbol in unique:
                results.append(legend[legend["Symbol"]==two_char_symbol].index[0])
                split.append([two_char_symbol])
                i += 2  # Move past this two-character symbol
                continue

        # Single character processing
        if input_string[i] in both or input_string[i] in suffix_list:
            results.append(legend[legend["Symbol"]==input_string[i]].index[0])
            split.append([input_string[i]])   

        i += 1  # Move to the next character

    return results,split


In [9]:
from copy import deepcopy

df_ = deepcopy(df)
del df
for column in df_.columns[2:]:
    for index, row in df_.iterrows():
        df_.at[index, column], _ = process_symbols(row[column])


In [10]:
# Example use
input_string = "CCDREV"
result, split = process_symbols(input_string)
print(f"Result for '{input_string}': {result} - Split: {split}")

input_string = "CCSDWEV"
result, split = process_symbols(input_string)
print(f"Result for '{input_string}': {result} - Split: {split}")

input_string = "—"
result, split = process_symbols(input_string)
print(f"Result for '{input_string}': {result} - Split: {split}")

input_string = "CCT"
result, split = process_symbols(input_string)
print(f"Result for '{input_string}': {result} - Split: {split}")

Result for 'CCDREV': [8, 13, 10] - Split: [['CC'], ['DR'], ['EV']]
Result for 'CCSDWEV': [8, 1, 14, 10] - Split: [['CC'], ['S'], ['DW'], ['EV']]
Result for '—': [18] - Split: [['—']]
Result for 'CCT': [8, 17] - Split: [['CC'], ['T']]


In [11]:
df_.set_index(('Name', '', 'Name'), inplace=True)

# Replace unicode characters in the index
df_.index = df_.index.str.replace('\u2640', 'f').str.replace('\u2642', 'm').str.lower()

# Remove special characters and lowercase the index
df_.index = df_.index.str.replace('[^A-Za-z0-9]+', '', regex=True)

In [12]:
# Custom aggregation function to concatenate lists
def concatenate_lists(series):
    if series.dtype == int:
        return series[0]
    elif series.dtype == list:
        return list(set(sum(series, [])))

# Group by index and apply the custom aggregation function
df_ = df_.groupby(df_.index).agg(concatenate_lists)


/var/folders/67/9rpqzr212xbc567t79k0w7580000gn/T/ipykernel_67115/2975098677.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return series[0]


In [13]:
df_.to_json("data/pokemon_availability.json", orient="index", indent=4)

In [14]:
legend.to_json("data/availability_legend.json", orient="index", indent=4)

In [15]:
games = []
for columns in df_.columns[1:]:
    games.append(columns[2])
games = pd.DataFrame(games, columns=["Game"])
games.to_json("data/games.json", orient="index", indent=4)

In [16]:
game_availability = pd.read_json("data/pokemon_availability.json", orient="index")
legend = pd.read_json("data/availability_legend.json", orient="index")
game_availability = game_availability.applymap(
    lambda x: (
        reduce(operator.or_, [legend.loc[i, "Availability"] for i in x])
        if isinstance(x, list)
        else x
    )
)

if "Concatenated" in game_availability.columns:
    game_availability.drop(columns="Concatenated", inplace=True)
game_availability["Concatenated"] = game_availability.apply(
    lambda row: [sublist for sublist in row[1:]], axis=1
)
game_availability.drop(columns=[col for col in game_availability.columns if col not in ["Concatenated", "('#', '', '#')"]], inplace=True)
game_availability.columns = ["#", "Values"]
game_availability.to_json("data/postproc_game_av.json", orient="index", indent=4)

/var/folders/67/9rpqzr212xbc567t79k0w7580000gn/T/ipykernel_67115/1274798945.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  game_availability = game_availability.applymap(
